In [24]:
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import TextLoader, DirectoryLoader
from langchain.agents import create_pandas_dataframe_agent
from langchain.agents import create_csv_agent
from langchain.chains import RetrievalQA
from dotenv import load_dotenv
import json
import os
import pandas as pd

#!pip install unstructured tiktoken

In [25]:
os.environ["OPENAI_API_KEY"] = "" # should use .env and load_dotenv
llm = OpenAI(openai_api_key = os.environ["OPENAI_API_KEY"], temperature = 0)

In [26]:
# inspect csv

In [27]:
tabular_data = pd.read_csv('../data/hack.csv')
tabular_data_agent = create_pandas_dataframe_agent(llm, tabular_data, verbose = True)

In [28]:
tabular_data.head()

,Date,Year,Category,Payee,Description,Location,Zip Code,Type,Trx Numbers,Amount,Unnamed: 10,Unnamed: 11
0,10/9/2022,2022,Revenue,Online Orders,Website Sales,Blue Pot 2,94020,Cash in,NaN,533.39,NaN,NaN
1,6/15/2022,2022,Expense,Utility Company,Electricity Bill,Blue Pot 2,94020,Cash out,NaN,583.80,NaN,NaN
2,12/18/2022,2022,Cost,Coffee Supplier,Coffee Beans Purchase,Blue Pot 1,94070,Cash out,NaN,524.79,NaN,NaN
3,9/14/2022,2022,Cost,Coffee Supplier,Coffee Beans Purchase,Blue Pot 1,94070,Cash out,NaN,83.16,NaN,NaN
4,6/19/2023,2023,Cost,Equipment Supplier,Espresso Machine Purchase,Blue Pot 7,94025,Cash out,NaN,593.46,NaN,NaN


In [29]:
for col in tabular_data.columns:
    print('-'*50)
    print(col)
    print(tabular_data[col].unique()[:5])

--------------------------------------------------
Date
['10/9/2022' '6/15/2022' '12/18/2022' '9/14/2022' '6/19/2023']
--------------------------------------------------
Year
[2022 2023]
--------------------------------------------------
Category
['Revenue' 'Expense' 'Cost']
--------------------------------------------------
Payee
['Online Orders' 'Utility Company' 'Coffee Supplier' 'Equipment Supplier'
 'Local Dairy']
--------------------------------------------------
Description
['Website Sales' 'Electricity Bill' 'Coffee Beans Purchase'
 'Espresso Machine Purchase' 'Milk and Cream Purchase']
--------------------------------------------------
Location
['Blue Pot 2' 'Blue Pot 1' 'Blue Pot 7' 'Blue Pot 4' 'Blue Pot 9']
--------------------------------------------------
Zip Code
[94020 94070 94025 94404 22180]
--------------------------------------------------
Type
['Cash in' 'Cash out']
--------------------------------------------------
Trx Numbers
[nan]
-------------------------------

In [21]:
tabular_data.shape

(5976, 12)

In [20]:
tabular_data_agent.run('How many rows are there in my data?')



> Entering new  chain...
Thought: I need to find out how many rows are in the dataframe
Action: python_repl_ast
Action Input: df.shape[0]
Observation: 5976
Thought: I now know the final answer
Final Answer: There are 5976 rows in my data.

> Finished chain.


'There are 5976 rows in my data.'

In [22]:
tabular_data_agent.run('What is my highest expense in zip code 94020?')



> Entering new  chain...
Thought: I need to find the highest expense in zip code 94020
Action: python_repl_ast
Action Input: df[(df['Zip Code'] == 94020) & (df['Type'] == 'Cash out')]['Amount'].max()
Observation: 20000.0
Thought: I now know the final answer
Final Answer: The highest expense in zip code 94020 is 20000.0.

> Finished chain.


'The highest expense in zip code 94020 is 20000.0.'

In [14]:
tabular_data_agent.run('What is the description of my most recent revenue? Include the top 5 with description and amount as a table.')



> Entering new  chain...
Thought: I need to sort the dataframe by date and then filter for revenue.
Action: python_repl_ast
Action Input: df.sort_values(by='Date', ascending=False).head()[df['Category'] == 'Revenue']
Observation:           Date  Year Category           Payee Description    Location  \
2048  9/9/2022  2022  Revenue  In-Store Sales  Hot Coffee  Blue Pot 5   

      Zip Code     Type  Trx Numbers  Amount  Unnamed: 10  Unnamed: 11  
2048     94401  Cash in          NaN  9423.7          NaN          NaN  
Thought:

<string>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


 I need to repeat this for the next 4 rows.
Action: python_repl_ast
Action Input: df.sort_values(by='Date', ascending=False).head()[df['Category'] == 'Revenue'].iloc[1:]
Observation: Empty DataFrame
Columns: [Date, Year, Category, Payee, Description, Location, Zip Code, Type, Trx Numbers, Amount, Unnamed: 10, Unnamed: 11]
Index: []
Thought:

<string>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


 I now know the final answer
Final Answer: There are no other revenue entries in the top 5.

> Finished chain.


'There are no other revenue entries in the top 5.'

In [22]:
tabular_data_agent.run('What are my biggest expenses? Please give a list by the descriptions.')



> Entering new  chain...
Thought: I need to sort the dataframe by the amount column.
Action: python_repl_ast
Action Input: df.sort_values(by='Amount', ascending=False)
Observation:             Date  Year Category               Payee  \
7      5/21/2023  2023  Revenue      In-Store Sales   
44     10/6/2022  2022  Revenue      In-Store Sales   
3741    5/6/2023  2023     Cost         Local Dairy   
4977    5/5/2023  2023     Cost         Local Dairy   
4378   6/16/2023  2023     Cost         Local Dairy   
...          ...   ...      ...                 ...   
2707   5/19/2022  2022     Cost     Coffee Supplier   
1902   12/3/2022  2022     Cost  Equipment Supplier   
2175  10/10/2022  2022     Cost         Local Dairy   
3      9/14/2022  2022     Cost     Coffee Supplier   
5      11/1/2022  2022     Cost         Local Dairy   

                    Description     Location  Zip Code      Type  Trx Numbers  \
7                   Iced Coffee   Blue Pot 1     94070   Cash in          N

'Milk and Cream Purchase, Iced Coffee, Espresso Machine Purchase, Coffee Beans Purchase.'

In [4]:
# general Q&A on existing data

In [30]:
# get existing embeddings
embeddings = OpenAIEmbeddings()
loader = DirectoryLoader("../data", glob="*.txt")
documents = loader.load()

# split text into chunks
text_splitter = CharacterTextSplitter(chunk_size = 200,chunk_overlap = 10)
texts = text_splitter.split_documents(documents)

# store in Chroma, texts converted to embeddings within
vecstore = Chroma.from_documents(texts, embeddings)

# define retriever
q_and_a = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type = "stuff",
    retriever = vecstore.as_retriever()
)

def query(q):
    print("Question : ", q)
    print("Answer   : ", q_and_a(q))

In [21]:
query('What are my biggest expenses? Please give a list by the descriptions.')

Question :  What are my biggest expenses? Please give a list by the descriptions.


Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


Answer   :  {'query': 'What are my biggest expenses? Please give a list by the descriptions.', 'result': " I don't know."}


In [32]:
# try to understand what it takes to create a dialogue where the user input is added to the embedding DB

In [31]:
# example from the web

# Step 3: Get user input
user_input = input("Enter a concept: ")
 
# Step 4: Define the Prompt Template
prompt = PromptTemplate(
    input_variables=["concept"],
    template="Define {concept} with a real-world example?",
)
 
# Step 5: Print the Prompt Template
print(prompt.format(concept=user_input))
 
# Step 6: Instantiate the LLMChain
#llm = OpenAI(temperature=0.9) # use the llm from above
chain = LLMChain(llm=llm, prompt=prompt)
 
# Step 7: Run the LLMChain
output = chain.run(user_input)
print(output)

Enter a concept:  What is my biggest expense?


Define What is my biggest expense? with a real-world example?


Your biggest expense will depend on your individual lifestyle and financial situation. For example, if you own a home, your biggest expense may be your mortgage payment. If you rent, your biggest expense may be your rent payment. Other common expenses include car payments, insurance premiums, utilities, groceries, and credit card payments.


In [ ]:
# how to expose an endpoint?